In [232]:
import xgboost
import pandas as pd
import sklearn.metrics as metr
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor

In [233]:
data = pd.read_csv('data/reformatted_data.csv').drop(['Unnamed: 0', 'CITY', 'ADDRESS', 'REGION', 'coords'], axis=1)
data = data[data.hh_500 > 50]

In [234]:
postamats = data[data.Postamat_daily == data.Postamat_daily].copy()
cashboxes = data[data.cashbox_daily == data.cashbox_daily].copy()

In [235]:
postamats.drop(['cashbox_daily', 'cashbox_MAX', 'cashbox_trend', 'Postamat_trend'], axis=1, inplace=True)

In [236]:
cashboxes.drop(['Postamat_daily', 'Postamat_trend', 'cashbox_trend'], axis=1, inplace=True)

In [237]:
print(postamats.isna().any().any())
print(cashboxes.isna().any().any())

False
False


In [238]:
cashboxes.columns

Index(['cashbox_daily', 'cashbox_MAX', 'hh_500', 'value1', 'value2',
       'POPULATION', 'NEAR_Malls', 'NEAR_Business_centers', 'NEAR_metro_rjd',
       'macro_salary_avg_yearly', 'Avg_check', 'dist_to_closest_shop',
       'dist_to_closest_postomat', 'dist_to_competitor', 'dist_to_post',
       'NEAR_shops', 'NEAR_stops'],
      dtype='object')

In [239]:
X_p = postamats[['hh_500', 'value1', 'value2', 'POPULATION',
       'NEAR_Malls', 'NEAR_Business_centers', 'NEAR_metro_rjd',
       'macro_salary_avg_yearly', 'Avg_check', 'dist_to_closest_shop',
       'dist_to_closest_postomat', 'dist_to_competitor', 'dist_to_post',
       'NEAR_shops', 'NEAR_stops', 'NEAR_shops']]
y_p = postamats[['Postamat_daily']]

In [240]:
X_c = cashboxes[['hh_500', 'value1', 'value2',
       'POPULATION', 'NEAR_Malls', 'NEAR_Business_centers', 'NEAR_metro_rjd',
       'macro_salary_avg_yearly', 'Avg_check', 'dist_to_closest_shop',
       'dist_to_closest_postomat', 'dist_to_competitor', 'dist_to_post',
       'NEAR_shops', 'NEAR_stops']]
y_c = cashboxes[['cashbox_daily', 'cashbox_MAX']]

In [241]:
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_p, y_p)
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_c, y_c)

In [242]:
xgb_p = xgboost.XGBRegressor(objective='reg:squarederror', max_depth=4, learning_rate=0.1, subsample=0.85, gamma=0.1, n_estimators=500)
xgb_p.fit(X_train_p, y_train_p)
metr.mean_squared_error(xgb_p.predict(X_test_p), y_test_p)

1.3814178129818113

In [243]:
xgb_c = MultiOutputRegressor(xgboost.XGBRegressor(objective='reg:squarederror', max_depth=4,
                                                             learning_rate=0.1, subsample=0.85, 
                                                             gamma=0.1, n_estimators=500)).fit(X_train_c, y_train_c)
xgb_c.predict(X_test_c)
metr.mean_squared_error(xgb_c.predict(X_test_c), y_test_c)

5.014646646195693

Работает нормально - обучим на всех данных

In [244]:
xgb_p = xgboost.XGBRegressor(objective='reg:squarederror', max_depth=4, learning_rate=0.1, subsample=0.85, gamma=0.1, n_estimators=500)
xgb_p.fit(X_p, y_p)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.1,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=4, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.85, verbosity=1)

In [245]:
xgb_c = MultiOutputRegressor(xgboost.XGBRegressor(objective='reg:squarederror', max_depth=4,
                                                             learning_rate=0.1, subsample=0.85, 
                                                             gamma=0.1, n_estimators=500)).fit(X_c, y_c)

In [246]:
postamats_target = data[data.Postamat_daily != data.Postamat_daily].copy()
cashboxes_target = data[data.cashbox_daily != data.cashbox_daily].copy()

In [247]:
postamats_target.columns

Index(['Postamat_daily', 'cashbox_daily', 'Postamat_trend', 'cashbox_MAX',
       'cashbox_trend', 'hh_500', 'value1', 'value2', 'POPULATION',
       'NEAR_Malls', 'NEAR_Business_centers', 'NEAR_metro_rjd',
       'macro_salary_avg_yearly', 'Avg_check', 'dist_to_closest_shop',
       'dist_to_closest_postomat', 'dist_to_competitor', 'dist_to_post',
       'NEAR_shops', 'NEAR_stops'],
      dtype='object')

In [248]:
pred_p = xgb_p.predict(postamats_target.drop(['Postamat_daily', 'cashbox_daily', 'Postamat_trend',
                                              'cashbox_MAX', 'cashbox_trend'], axis=1))
postamats_target['Postamat_daily'] = pred_p

ValueError: feature_names mismatch: ['hh_500', 'value1', 'value2', 'POPULATION', 'NEAR_Malls', 'NEAR_Business_centers', 'NEAR_metro_rjd', 'macro_salary_avg_yearly', 'Avg_check', 'dist_to_closest_shop', 'dist_to_closest_postomat', 'dist_to_competitor', 'dist_to_post', 'NEAR_stops', 'NEAR_shops'] ['hh_500', 'value1', 'value2', 'POPULATION', 'NEAR_Malls', 'NEAR_Business_centers', 'NEAR_metro_rjd', 'macro_salary_avg_yearly', 'Avg_check', 'dist_to_closest_shop', 'dist_to_closest_postomat', 'dist_to_competitor', 'dist_to_post', 'NEAR_shops', 'NEAR_stops']

In [210]:
pred_c = xgb_c.predict(cashboxes_target.drop(['cashbox_daily', 'cashbox_MAX', 'Postamat_daily', 'Postamat_trend', 'cashbox_trend'], axis=1))

In [249]:
cashboxes_target['cashbox_daily'] = pred_c[:, 0]
cashboxes_target['cashbox_MAX'] = pred_c[:, 1]

In [250]:
postamats_target.to_csv('data/postamats_predicted.csv')

In [251]:
cashboxes_target.to_csv('data/cashboxes_predicted.csv')